In [ ]:
%loadFromPOM pom.xml
%classpath C:\\Project\\HAPI-fhir-Lab

In [ ]:
import ca.uhn.fhir.context.FhirContext;
import ca.uhn.fhir.rest.client.api.*;
import ca.uhn.fhir.rest.api.MethodOutcome;
import ca.uhn.fhir.validation.*;
import org.hl7.fhir.r4.model.*;
import org.hl7.fhir.common.hapi.validation.validator.*;
import org.hl7.fhir.common.hapi.validation.support.*;
import ca.uhn.fhir.context.support.*;
import ca.uhn.fhir.util.*;
import org.hl7.fhir.instance.model.api.*;
import java.nio.file.*;

In [ ]:
FhirContext ctx = FhirContext.forR4();
String serverBase = "http://localhost:8080/fhir";
IGenericClient client = ctx.newRestfulGenericClient(serverBase);

In [ ]:
// Create a validator and register it with the context
FhirValidator validator = ctx.newValidator();

// Create a validation module and register itwith the validator
//IValidatorModule module = new FhirInstanceValidator(ctx);

// Register the module with the validator
//validator.registerValidatorModule(module);

NpmPackageValidationSupport npmPackageSupport = new NpmPackageValidationSupport(ctx);
npmPackageSupport.loadPackageFromClasspath("classpath:profiles/tw-core-ig-v0.1.1.tgz");

// Create a validation support chain
ValidationSupportChain validationSupportChain = new ValidationSupportChain(
    npmPackageSupport,
    new DefaultProfileValidationSupport(ctx),
    new InMemoryTerminologyServerValidationSupport(ctx),
    new CommonCodeSystemsTerminologyService(ctx),
    new SnapshotGeneratingValidationSupport(ctx));

// Set the chain on the validator
FhirInstanceValidator instanceValidator = new FhirInstanceValidator(validationSupportChain);
validator.registerValidatorModule(instanceValidator);

In [24]:
Map<String, String> resourceProfiles = new HashMap<>();
resourceProfiles.put("Organization", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Organization-hosp-twcore");
resourceProfiles.put("Patient", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Patient-twcore");
resourceProfiles.put("Observation", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Observation-vitalSigns-twcore");
resourceProfiles.put("MedicationRequest", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/MedicationRequest-twcore");
resourceProfiles.put("Medication", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Medication-twcore");
resourceProfiles.put("Encounter", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Encounter-twcore");
resourceProfiles.put("Condition", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Condition-twcore");
resourceProfiles.put("DiagnosticReport", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/DiagnosticReport-twcore");
resourceProfiles.put("Procedure", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Procedure-twcore");
resourceProfiles.put("Practitioner", "https://twcore.mohw.gov.tw/ig/twcore/StructureDefinition/Practitioner-twcore");

// Read JSON-encoded FHIR resource from file
String filePath = "C:\\Project\\HAPI-fhir-Lab\\synthetic-data\\fhir\\hospitalInformation.json";
String json = new String(Files.readAllBytes(Paths.get(filePath)));

// Parse JSON-encoded FHIR resource
Bundle bundle = ctx.newJsonParser().parseResource(Bundle.class, json);

// From Bundle, read data
ValidationResult result = null;
String resourceType = null;

// Create a new bundle
Bundle outBundle = new Bundle(); 
outBundle.setType(Bundle.BundleType.TRANSACTION);

for (Bundle.BundleEntryComponent entry : bundle.getEntry()) {
    result = null;
    resourceType = entry.getResource().getResourceType().name();

    DomainResource resource = (DomainResource) entry.getResource();
    
    resource.getMeta().addProfile(resourceProfiles.get(resourceType));
    result = validator.validateWithResult(resource);
    while (result != null && !result.isSuccessful()) {
        if (resourceType == "Organization") {
            Organization org = (Organization) resource;
            Identifier id = new Identifier();
            org.setIdentifier(null);
            org.setExtension(null);
            id.setSystem("https://twcore.mohw.gov.tw/ig/twcore/CodeSystem/organization-identifier-tw");
            id.setValue(org.getName());
            org.addIdentifier(id);
            resource = org;
        }
        else if(resourceTypes == "Patient"){
            Patient patient = (Patient) resource;
            Identifier id = new Identifier();
            
        }      
        result = validator.validateWithResult(resource);
    }
    
    outBundle.addEntry()
        .setFullUrl(resource.getIdElement().getValue())
        .setResource(resource)
        .getRequest()
        .setUrl(resourceType)
        .setMethod(Bundle.HTTPVerb.PUT);
}

//serialize the bundle to output json filr "C:\\Project\\HAPI-fhir-Lab\\synthetic-out\\fhir\\hospitalInformation-out.json"
String outJson = ctx.newJsonParser().setPrettyPrint(true).encodeResourceToString(bundle);

// Write JSON-encoded FHIR resource to file
String outFilePath = "C:\\Project\\HAPI-fhir-Lab\\synthetic-data\\fhir\\hospitalInformation-out.json";
Files.write(Paths.get(outFilePath), outJson.getBytes());

C:\Project\HAPI-fhir-Lab\synthetic-data\fhir\hospitalInformation-out.json